In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import re
import collections
import json

In [ ]:
train = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv')

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.drop_duplicates('cleaned_label')

In [ ]:
train[train.Id=='170113f9-399c-489e-ab53-2faf5c64c5bc']

In [ ]:
test = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/sample_submission.csv')

In [ ]:
test.head()

In [ ]:
test.describe()

In [ ]:
with open('/kaggle/input/coleridgeinitiative-show-us-the-data/train/93c27c16-9356-4b97-9785-9795ad9bb609.json') as json_file:
    t = json.load(json_file)
    text = []
    for p in t:
        text.append(p['text'])
    text = ' '.join(text)

In [ ]:
t

In [ ]:
text

In [ ]:
train[train.Id=='93c27c16-9356-4b97-9785-9795ad9bb609']

In [ ]:
%%time
texts = {}
for i, id in enumerate(train.Id.unique()):
    print(i, end=' ') if i%100==0 else next
    with open('/kaggle/input/coleridgeinitiative-show-us-the-data/train/'+id+'.json') as json_file:
        t = json.load(json_file)
        text = []
        for p in t:
            text.append(p['text'])
        text = ' '.join(text)
        texts[id] = text

In [ ]:
all_texts = ' '.join(list(texts.values()))

In [ ]:
%%time
test_texts = {}
for i, id in enumerate(test.Id.unique()):
    print(i, end=' ') if i%100==0 else next
    with open('/kaggle/input/coleridgeinitiative-show-us-the-data/test/'+id+'.json') as json_file:
        t = json.load(json_file)
        test_text = []
        for p in t:
            test_text.append(p['text'])
        test_text = ' '.join(test_text)
        test_texts[id] = test_text

In [ ]:
%%time
c = collections.Counter()
words = ' '.join(train.dataset_title.unique().tolist()).split() + ' '.join(train.dataset_label.unique().tolist()).split() + ' '.join(train.cleaned_label.unique().tolist()).split()
total_count = 0
for word in words:
    c[word.lower()] += 1
    total_count += 1

In [ ]:
print(total_count)
c.most_common(30)

In [ ]:
%%time
c_all = collections.Counter()
words = all_texts.split()
total_count_all = 0
for word in words:
    c_all[word.lower()] += 1
    total_count_all += 1

In [ ]:
print(total_count_all)
c_all.most_common(30)

In [ ]:
for word in c.most_common(30):
    print(word[0], '\t', round(word[1]/total_count,6), '\t', round(dict(c_all)[word[0]]/total_count_all,6))

In [ ]:
train['text'] = train['Id'].map(texts)
test['text'] = test['Id'].map(test_texts)

In [ ]:
train.head(2)

In [ ]:
test.head(2)

In [ ]:
%%time
train['cleaned_label_isin'] = train.apply(lambda x: 1 if x['text'].lower().find(x['cleaned_label'].lower()) else 0, axis=1)
train['dataset_label_isin'] = train.apply(lambda x: 1 if x['text'].lower().find(x['dataset_label'].lower()) else 0, axis=1)
train['dataset_title_isin'] = train.apply(lambda x: 1 if x['text'].lower().find(x['dataset_title'].lower()) else 0, axis=1)

In [ ]:
train.head(10)

In [ ]:
train[['cleaned_label_isin', 'dataset_label_isin', 'dataset_title_isin']].mean(axis=0)

In [ ]:
datawords = ['Survey', 'Data', 'Study', 'Database', 'Statistics']
t = 3000
print(train.loc[t,'cleaned_label'])
print('='*50)
for dataword in datawords:
    inds = [m.start() for m in re.finditer(dataword, train.loc[t, 'text'])]
    for i in inds:
        print(i, ':', train.loc[t, 'text'][i-40:i+40].replace('\n',' '))

In [ ]:
ds = list(set(train.dataset_title.unique().tolist() + train.dataset_label.unique().tolist() + train.cleaned_label.unique().tolist()))
ds = list(set([i.lower() for i in ds]))
print(len(ds))

In [ ]:
%%time
[x for x in ds if x in train.loc[1, 'text'].lower()]

In [ ]:
[d for d in ds if d.find('research')>-1]

In [ ]:
[d for d in ds if d.find('center')>-1]

In [ ]:
[d for d in ds if d.find('dataset')>-1]

In [ ]:
def clean_paper_sentence(s):
    """
    This function is essentially clean_text without lowercasing.
    """
    s = re.sub('[^A-Za-z0-9]+', ' ', str(s)).strip()
    s = re.sub(' +', ' ', s)
    return s

In [ ]:
%%time
datawords = ['Survey', 'Data', 'Study', 'Database', 'Statistics', 'Dataset']
train['good_candidates'] = ''
train['bad_candidates'] = ''
verbose = True
preps = ['of', 'and', 'in', 'for']

for t in range(len(train)):
    print('='*80) if verbose else next
    print('='*80) if verbose else next
    print(t, end=' ') #if t%100==0 and not verbose else print(t) if verbose else next
    #print(t, end=' ') if t%100==0 else next
    print(train.loc[t,'cleaned_label']) if verbose else next
    print('='*50) if verbose else next
    text = train.loc[t, 'text']
    text = re.sub("[\(\[].*?[\)\]]", "", text) #remove text in brackets
    text = text.replace('.', ' dot').replace(',',' comma') #replacing dots and commas
    text = text.replace('/n', ' paragraph ')
    text_words = re.sub(r"[^a-zA-Z0-9] "," ",text).split(' ')
    text_words = [w for w in text_words if w!='']
    good_candidates = []
    bad_candidates = []

    for dataword in datawords:
        inds = [i for i, w in enumerate(text_words) if w==dataword]
        for i in inds:
            print(i, ':', text_words[i-4:i+4]) if verbose else next
            lower_limit=i
            upper_limit=i

            #for j in range(1,6):
            #    if i-j>=0:
            #        if text_words[i-j][0].isupper() or text_words[i-j] in preps:
            #            lower_limit = i-j
            #        else:
            #            break
            
            for j in range(1,6):
                if i-j >= 0:
                    if text_words[i-j][0].isupper():
                        lower_limit = i-j
                    elif i-j-1 >= 0:
                        if text_words[i-j] in preps and text_words[i-j-1][0].isupper():
                            lower_limit = i-j-1
                        else:
                            break
                    else:
                        break
                else:
                    break

            for j in range(1,6):
                if i+j <len(text_words):
                    if text_words[i+j][0].isupper():
                        upper_limit = i+j
                    elif i+j+1 <len(text_words):
                        if text_words[i+j] in preps and text_words[i+j+1][0].isupper():
                            upper_limit = i+j+1
                        else:
                            break
                    else:
                        break
                else:
                    break
            
            if text_words[lower_limit] == 'The':
                lower_limit = lower_limit + 1
                
            #if text_words[lower_limit][-3:] =='ing':
            #    lower_limit = lower_limit + 1
                
            if 'Center' in text_words[lower_limit:upper_limit+1] or 'Source' in text_words[lower_limit:upper_limit+1] or 'Note' in text_words[lower_limit:upper_limit+1]:
                upper_limit=lower_limit
                
            if len([m.start() for m in re.finditer('Study', ' '.join(text_words[lower_limit:upper_limit+1]))])>1:
                upper_limit=lower_limit
            
            
            candidate = ' '.join(text_words[lower_limit:upper_limit+1])
            if len(candidate.split(' '))>2:
                good_candidates.append(candidate)
            else:
                bad_candidates.append(candidate)
                
            #if len(good_candidates) >=3:
            #    good_candidates = sorted(good_candidates, key=len)[-3:]

            print(candidate) if verbose else next
            print('='*30) if verbose else next
    
    text = train.loc[t, 'text']
    #text = re.sub("[\(\[].*?[\)\]]", "", text) #remove text in brackets
    text = text.replace('.', ' dot').replace(',',' comma') #replacing dots and commas
    text = text.replace('/n', ' paragraph ')
    text = clean_paper_sentence(text)
    
    dict_cands = [x for x in ds if x in text.lower()]
    for candidate in dict_cands:
        print(candidate) if verbose else next
        print('='*30) if verbose else next
        #if len(candidate.split(' '))>2:
        good_candidates.append(candidate)
        #else:
        #    bad_candidates.append(candidate)
    
    good_candidates = [i.lower() for i in good_candidates]
    bad_candidates = [i.lower() for i in bad_candidates]
    
    good_candidates = '|'.join(list(set(good_candidates)))
    bad_candidates = '|'.join(list(set(bad_candidates)))
    print('Good candidates: ', good_candidates) if verbose else next
    print('Bad candidates: ', bad_candidates) if verbose else next
    train.loc[t, 'good_candidates'] = good_candidates
    train.loc[t, 'bad_candidates'] = bad_candidates
    
    if t==1000:
        break

In [ ]:
train.head()

In [ ]:
%%time
datawords = ['Survey', 'Data', 'Study', 'Database', 'Statistics', 'Dataset']
test['good_candidates'] = ''
#train['bad_candidates'] = ''
verbose = False
preps = ['of', 'and', 'in', 'for']

for t in range(len(test)):
    print('='*80) if verbose else next
    print('='*80) if verbose else next
    #print(t, end=' ')# if t%100==0 and not verbose else print(t) if verbose else next
    print(t, end=' ') if t%100==0 else next
    #print(test.loc[t,'cleaned_label']) if verbose else next
    print('='*50) if verbose else next
    text = test.loc[t, 'text']
    text = re.sub("[\(\[].*?[\)\]]", "", text) #remove text in brackets
    text = text.replace('.', ' dot').replace(',',' comma') #replacing dots and commas
    text = text.replace('/n', ' paragraph ')
    text_words = re.sub(r"[^a-zA-Z0-9] "," ",text).split(' ')
    text_words = [w for w in text_words if w!='']
    good_candidates = []
    bad_candidates = []

    for dataword in datawords:
        inds = [i for i, w in enumerate(text_words) if w==dataword]
        for i in inds:
            print(i, ':', text_words[i-4:i+4]) if verbose else next
            lower_limit=i
            upper_limit=i

            #for j in range(1,6):
            #    if i-j>=0:
            #        if text_words[i-j][0].isupper() or text_words[i-j] in preps:
            #            lower_limit = i-j
            #        else:
            #            break
            
            for j in range(1,6):
                if i-j >= 0:
                    if text_words[i-j][0].isupper():
                        lower_limit = i-j
                    elif i-j-1 >= 0:
                        if text_words[i-j] in preps and text_words[i-j-1][0].isupper():
                            lower_limit = i-j-1
                        else:
                            break
                    else:
                        break
                else:
                    break

            for j in range(1,6):
                if i+j <len(text_words):
                    if text_words[i+j][0].isupper():
                        upper_limit = i+j
                    elif i+j+1 <len(text_words):
                        if text_words[i+j] in preps and text_words[i+j+1][0].isupper():
                            upper_limit = i+j+1
                        else:
                            break
                    else:
                        break
                else:
                    break
            
            if text_words[lower_limit] == 'The':
                lower_limit = lower_limit + 1
                
            #if text_words[lower_limit][-3:] =='ing':
            #    lower_limit = lower_limit + 1
                
            if 'Center' in text_words[lower_limit:upper_limit+1] or 'Source' in text_words[lower_limit:upper_limit+1] or 'Note' in text_words[lower_limit:upper_limit+1]:
                upper_limit=lower_limit
                
            if len([m.start() for m in re.finditer('Study', ' '.join(text_words[lower_limit:upper_limit+1]))])>1:
                upper_limit=lower_limit
            
            
            candidate = ' '.join(text_words[lower_limit:upper_limit+1])
            if len(candidate.split(' '))>2:
                good_candidates.append(candidate)
            else:
                bad_candidates.append(candidate)
                
            #if len(good_candidates) >=3:
            #    good_candidates = sorted(good_candidates, key=len)[-3:]

            print(candidate) if verbose else next
            print('='*30) if verbose else next
    
    text = test.loc[t, 'text']
    #text = re.sub("[\(\[].*?[\)\]]", "", text) #remove text in brackets
    text = text.replace('.', ' dot').replace(',',' comma') #replacing dots and commas
    text = text.replace('/n', ' paragraph ')
    text = clean_paper_sentence(text)
    
    dict_cands = [x for x in ds if x in text.lower()]
    for candidate in dict_cands:
        print(candidate) if verbose else next
        print('='*30) if verbose else next
        #if len(candidate.split(' '))>2:
        good_candidates.append(candidate)
        #else:
        #    bad_candidates.append(candidate)
    
    good_candidates = [i.lower() for i in good_candidates]
    #bad_candidates = [i.lower() for i in bad_candidates]
    
    good_candidates = '|'.join(list(set(good_candidates)))
    #bad_candidates = '|'.join(list(set(bad_candidates)))
    #print('Good candidates: ', good_candidates) if verbose else next
    #print('Bad candidates: ', bad_candidates) if verbose else next
    test.loc[t, 'PredictionString'] = good_candidates
    #train.loc[t, 'bad_candidates'] = bad_candidates
    
    #if t==3000:
    #    break

In [ ]:
test[['Id', 'PredictionString']].to_csv('submission.csv', index=False)
test.head()